In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
img_filepath = "/content/drive/MyDrive/BTT Skinterest 2A/Dataset/images"
csv_filepath = "/content/drive/MyDrive/BTT Skinterest 2A/Dataset/SCIN Working Merged Spreadsheet.csv"

In [10]:
csv_df = pd.read_csv(csv_filepath)
csv_df.head()

,case_id,source,release,year,age_group,sex_at_birth,fitzpatrick_skin_type,dermatologist_fitzpatrick_skin_type_label_1,dermatologist_fitzpatrick_skin_type_label_2,dermatologist_fitzpatrick_skin_type_label_3,...,related_category,condition_duration,image_1_path,image_2_path,image_3_path,image_1_shot_type,image_2_shot_type,image_3_shot_type,combined_race,race_ethnicity_two_or_more_after_mitigation
0,-1.262950e+18,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NaN,FST3,0,0,...,NaN,NaN,dataset/images/-5949315841433628424.png,dataset/images/-8183947049312687778.png,dataset/images/4923188439600899486.png,AT_DISTANCE,CLOSE_UP,AT_AN_ANGLE,NaN,NaN
1,-1.489550e+18,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NaN,FST2,0,0,...,NaN,NaN,dataset/images/325464533153467313.png,NaN,NaN,CLOSE_UP,NaN,NaN,NaN,NaN
2,-1.492320e+18,SCIN,1.0.0,2023,AGE_18_TO_29,FEMALE,NONE_IDENTIFIED,FST2,FST2,FST2,...,LOOKS_HEALTHY,ONE_TO_FOUR_WEEKS,dataset/images/-6837240536182868524.png,dataset/images/6395257111195214043.png,dataset/images/7877276387406078156.png,AT_DISTANCE,CLOSE_UP,AT_AN_ANGLE,HISPANIC_LATINO_OR_SPANISH_ORIGIN,NaN
3,-1.580900e+17,SCIN,1.0.0,2023,AGE_18_TO_29,FEMALE,FST3,FST3,0,0,...,RASH,ONE_TO_FOUR_WEEKS,dataset/images/2983323875335943836.png,NaN,NaN,CLOSE_UP,NaN,NaN,WHITE,NaN
4,-1.887630e+18,SCIN,1.0.0,2023,AGE_30_TO_39,MALE,FST3,FST2,0,0,...,OTHER_ISSUE_DESCRIPTION,LESS_THAN_ONE_WEEK,dataset/images/3104801012387799539.png,dataset/images/8660513260658813359.png,NaN,CLOSE_UP,AT_AN_ANGLE,NaN,WHITE,NaN
